In [ ]:
#importing necessary libraries

import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from warnings import warn
import time

In [ ]:
#passing required URL for scrapping

driver=webdriver.Chrome("chromedriver.exe")
driver.get("https://www.linkedin.com")

In [ ]:
#logging in using keys

inputID = driver.find_element(by=By.ID, value = "username")
inputPass = driver.find_element(by=By.ID, value = "password")
signIn = driver.find_element(by=By.CLASS_NAME, value = "login__form_action_container ")
inputID.send_keys("email_id")
inputPass.send_keys("password")
signIn.click()

time.sleep(10)

login_btn = driver.find_element(By.CLASS_NAME,"sign-in-form__submit-button")
login_btn.click()

In [ ]:
#redirecting to desired URL

driver.get("https://www.linkedin.com/jobs/collections/")

In [ ]:
#list of elements required

name = []
designation = []
location = []
job_link = []
industry = []
emp_count = []
linkedin_followers = []
applicants = []
involvement = []
work_type = []

In [ ]:
#iterating through page

for i in range(1,41):
    #button path for page numbers
    path ='//button[@aria-label="Page {}"]'.format(i)
    
    #button clicking    
    driver.find_element(By.XPATH, path).click()    
    
    #html data
    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')
    
    #main page of one job data
    lk=soup.findAll(class_="disabled ember-view job-card-container__link")
     
    #link of a single job data
    for i in lk:
        # links
        li=i['href']
        
        #every page data
        every_page =BeautifulSoup(driver.page_source,'lxml')
                
        #movig to link using next window_of_ chrome -- alternative of redirecting to original URL
        driver.switch_to.new_window('tab')
        job_link.append("https://www.linkedin.com{}".format(li))
        driver.get("https://www.linkedin.com{}".format(li))
        
       
         # company name
        try:
            c_name = driver.find_elements(By.CLASS_NAME,'jobs-unified-top-card__company-name')
            name.append(c_name[0].text) 
        except:
            name.append("N.A.")
         
        #designation   
        try:
            d = driver.find_elements(By.CLASS_NAME,'jobs-unified-top-card__job-title')
            designation.append(d[0].text)
        except:
            designation.append("N.A.")
        
        #applicants
        try:
            apl= driver.find_elements(By.XPATH,'/html/body/div[5]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/div[1]/span[2]/span[2]/span')
            applicants.append(apl[0].text) 
        except:
            applicants.append("0") 
            
            
            
        #work type
        try:
            w = driver.find_elements(By.CLASS_NAME,'jobs-unified-top-card__workplace-type')
            work_type.append(w[0].text)  
        except:
            work_type.append("N.A.")
        
        
        #involvement
        try:
            inv = driver.find_elements(By.CLASS_NAME,'jobs-unified-top-card__job-insight')
            involvement.append(inv[0].text)
        except:
            involvement.append("N.A.")
         
        #employee count
        try:
            emp = driver.find_elements(By.CLASS_NAME,'jobs-unified-top-card__job-insight')
            emp_count.append(emp[1].text)
        except:
            emp_count.append("N.A.")
        
        #location
        try:
            loc = driver.find_elements(By.CLASS_NAME,'jobs-unified-top-card__bullet')
            location.append(loc[0].text)
        except:
            location.append("N.A.")
    
        #every page data
        every_page =BeautifulSoup(driver.page_source,'lxml')
        
        
        # details
        s = []
        src = driver.page_source
        soup = BeautifulSoup(src, 'lxml')
        detail = soup.findAll(class_='ember-view t-black t-normal')
        for z in detail:
            s.append(z)

        # selecting new jobs 
        for i in s:
            pr = i['href']
            
            #movig to link using next window_of_ chrome
            driver.switch_to.new_window('tab')
            driver.get("https://www.linkedin.com{}".format(pr))
            
            time.sleep(6)
            
            #industry
            try:
                ind = driver.find_elements(By.CLASS_NAME,'org-top-card-summary-info-list__info-item')
                industry.append(ind[0].text)
            except:
                industry.append("not specify")

            #followers    
            try:
                follow = driver.find_elements(By.XPATH,'//*[@id="ember28"]/div[2]/div[1]/div[1]/div[2]/div/div/div[2]/div[2]')
                linkedin_followers.append(follow[0].text)
            except:
                linkedin_followers.append("N/A")
                
                
            #close current window       
            driver.close()
            #switch to main(starting) tab/window
            driver.switch_to.window(driver.window_handles[-1])
     
               
        # close current window       
        driver.close()
        #switch to main (starting) tab/window
        driver.switch_to.window(driver.window_handles[0])


In [ ]:
#checking length of lists 

len(name), len(location), len(applicants), len(designation),len(emp_count),len(industry),len(linkedin_followers),len(involvement),len(work_type)

In [ ]:
#creating tables using pandas

main_table = pd.DataFrame({'name':name,'employees_count':emp_count,'linkedin_followers':linkedin_followers,'industry':industry,involvement':involvement,'work_type':work_type ,'total_applicants':applicants})


In [ ]:
import openpyxl

main_table.to_excel('main_table.xlsx', sheet_name='sheet_1')